# Prepare Data

***Import libraries required***

In [ ]:
!pip install tensorflow

In [1]:
# import libraries for file utilities
import os
import zipfile

In [2]:
# import standard ml libraries 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# tensorflow/core/util/util.cc:169] oneDNN custom operations are on by default
# May result in slightly different numerical results due to floating-point round-off errors from different computation orders. 
# Setting the environment variable TF_ENABLE_ONEDNN_OPTS=0 to turn them off.
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [4]:
import tensorflow as tf
from tensorflow import keras

2024-01-16 22:42:00.742961: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [5]:
# library to load images
from tensorflow.keras.preprocessing.image import load_img

In [6]:
# library to import Xception CNN
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

## Import

In [ ]:
# copy data zip file to /data folder and unzip it 
zip_path = './data/catbreeds_data.zip'
extract_path = './data/'
# Check if the folder already exists
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print("Data files extracted successfully from zip file.")
else:
    print("Folder already exists. Skipping data extraction.")

## Explore

In [ ]:
# check that an image can be loaded
path = './data/train/Abyssinian'
name = 'Abyssinian_100.jpg'
fullname = f'{path}/{name}'
img = load_img(fullname, target_size=(299, 299))
img

In [ ]:
x = np.array(img)
x.shape

## Split

# Build Model

## Baseline

***Use Xception model as baseline***

In [ ]:
model = Xception(weights='imagenet', input_shape=(299, 299, 3))

In [ ]:
X = np.array([x])

In [ ]:
X.shape

In [ ]:
X = preprocess_input(X)
X[0]

In [ ]:
pred = model.predict(X)

In [ ]:
pred.shape

In [ ]:
decode_predictions(pred)

***Load images and train models***

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# run model using train data
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_ds = train_gen.flow_from_directory(
    './data/train/',
    target_size=(150, 150),
    batch_size=32
)

In [ ]:
# display classes
train_ds.class_indices

In [ ]:
# run model using validation data
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    './data/validation/',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

In [ ]:
train_ds.class_indices

***Create base model***

In [ ]:
# create convolutional layers
base_model = Xception(weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)

# set trainable to flase as we want to use existing model
base_model.trainable = False

# define model layers
inputs = keras.Input(shape=(150, 150, 3))

base = base_model(inputs, training=False)

vectors = keras.layers.GlobalAveragePooling2D() (base)

outputs = keras.layers.Dense(20) (vectors)

model = keras.Model(inputs, outputs)

In [ ]:
# Set-up X (cat whose breeds needs to be identified)
path = './data/train/Abyssinian'
name = 'Abyssinian_100.jpg'
fullname = f'{path}/{name}'
img = load_img(fullname, target_size=(150, 150))
x = np.array(img)
X = np.array([x])
X.shape

In [ ]:
preds = model.predict(X)

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
base_history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
# plot training accuracy vs validation accuracy
plt.plot(base_history.history['accuracy'], label='train')
plt.plot(base_history.history['val_accuracy'], label='val')

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# plot training loss vs validation loss
plt.plot(base_history.history['loss'], label='loss')
plt.plot(base_history.history['val_loss'], label='val_loss')

plt.xticks(np.arange(10))
plt.legend()

## Iterate

***Iteration 1: Adjust the learning rate***

In [ ]:
def make_model(learning_rate):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

    base_model.trainable = False

    # define model architecture
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    outputs = keras.layers.Dense(20)(vectors)
    model = keras.Model(inputs, outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
scores = {}
for lr in [0.0001, 0.001, 0.01, 0.1]:
    print(lr)
    
    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history
    
    print()
    print()
    

In [ ]:
for lr, hist in scores.items():
    plt.plot(hist['accuracy'], label=lr)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# plot training loss 
for lr, hist in scores.items():
    plt.plot(hist['loss'], label=lr)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
for lr, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=lr)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# plot training loss 
for lr, hist in scores.items():
    plt.plot(hist['val_loss'], label=lr)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
learning_rate = 0.001
print(f"Best learning rate: {learning_rate}")

***Iteration 2: implement callbacks to save best model***

In [ ]:
model.save_weights('model_v1.h5', save_format='h5')

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v2_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)   

In [ ]:
model = make_model(learning_rate=learning_rate)

history = model.fit(
    train_ds, 
    epochs=10, 
    validation_data=val_ds
#    callbacks=[checkpoint]
)

***Iteration 3: Add more layers to model***

In [ ]:
def make_model(learning_rate, size_inner):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

    base_model.trainable = False

    # define model architecture
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    outputs = keras.layers.Dense(20)(inner)
    model = keras.Model(inputs, outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v3_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001

scores = {}

for size in [10, 100, 500, 1000]:
    print(size)
    
    model = make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(
        train_ds, 
        epochs=10, 
        validation_data=val_ds 
#        callbacks=[checkpoint]
    )
    scores[size] = history.history
    
    print()
    print()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['accuracy'], label=size)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# plot training loss 
for size, hist in scores.items():
    plt.plot(hist['loss'], label=size)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=size)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
scores[100]['val_accuracy']

In [ ]:
# plot training loss 
for size, hist in scores.items():
    plt.plot(hist['val_loss'], label=size)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
best_size = 1000
print(f"Best size for additional layer: {best_size}")

***Iteration 4: Dropout and regularisation***

In [ ]:
def make_model(learning_rate, size_inner, droprate=0.5):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

    base_model.trainable = False

    # define model architecture
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    outputs = keras.layers.Dense(20)(drop)
    model = keras.Model(inputs, outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
learning_rate = 0.001
size_inner = 1000

scores = {}

for droprate in [0.0, 0.2, 0.5, 0.8]:
    print(droprate)
    
    model = make_model(
        learning_rate=learning_rate, 
        size_inner=size,
        droprate=droprate
    )
    
    history = model.fit(
        train_ds, 
        epochs=30, 
        validation_data=val_ds 
#        callbacks=[checkpoint]
    )
    scores[droprate] = history.history
    
    print()
    print()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['accuracy'], label='val=%s' % droprate)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# plot training loss 
for droprate, hist in scores.items():
    plt.plot(hist['loss'], label='val=%s' % droprate)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['val_accuracy'], label='val=%s' % droprate)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# plot training loss 
for droprate, hist in scores.items():
    plt.plot(hist['val_loss'], label='val=%s' % droprate)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
# droprate = 1000
# print(f"Best size for additional layer: {best_size}")

In [ ]:
# # Unit test
# scores = {}

# for size in [10]:
#     print(size)
    
#     model = make_model(learning_rate=learning_rate, size_inner=size)
#     history = model.fit(
#         train_ds, 
#         epochs=2, 
#         validation_data=val_ds, 
#         callbacks=[checkpoint]
#     )
#     scores[size] = history.history
    
#     print()
#     print()

In [ ]:
scores

***Iteration 5: Save model created with best hyperparemeters*** 

In [ ]:
learning_rate = 0.001
size_inner = 1000
droprate = 0.8
epochs = 10

model = make_model(
    learning_rate=learning_rate, 
    size_inner=size,
    droprate=droprate
)
    
history = model.fit(
    train_ds, 
    epochs=epochs, 
    validation_data=val_ds, 
    callbacks=[checkpoint]
)

***Use the saved model***

In [ ]:
model = keras.models.load_model("xception_v3_10_0.609.h5")

In [ ]:
# run model using test data
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = val_gen.flow_from_directory(
    './data/test/',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

In [ ]:
# evaluate test data set using evaluate function
model.evaluate(test_ds)

In [ ]:
path = './data/test/Abyssinian'
name = 'Abyssinian_1.jpg'
fullname = f'{path}/{name}'
img = load_img(fullname, target_size=(150, 150))
img

In [ ]:
x = np.array(img)
X = np.array([x])
X.shape

In [ ]:
X = preprocess_input(X)

In [ ]:
pred = model.predict(X)

In [ ]:
classes = ['Abyssinian', 
 'American Bobtail', 
 'American Curl',
 'American Shorthair',
 'Bengal',
 'Birman',
 'Bombay',
 'British Shorthair',
 'Egyptian Mau',
 'Exotic Shorthair',
 'Maine Coon',
 'Manx',
 'Norwegian Forest',
 'Persian',
 'Ragdoll',
 'Russian Blue',
 'Scottish Fold',
 'Siamese',
 'Sphynx',
 'Turkish Angora']

In [ ]:
dict(zip(classes, np.round(pred[0], 4)))

In [ ]:
print('fin')

***Save model as TF-Lite model***

In [ ]:
model = keras.models.load_model("xception_v3_10_0.609.h5")

In [ ]:
!python -V

In [ ]:
tf.__version__

In [ ]:
path = './data/test/Abyssinian'
name = 'Abyssinian_1.jpg'
fullname = f'{path}/{name}'
img = load_img(fullname, target_size=(150, 150))
img

In [ ]:
x = np.array(img)
X = np.array([x])
X.shape

In [ ]:
X = preprocess_input(X)

In [ ]:
preds = model.predict(X)

In [ ]:
classes = ['Abyssinian', 
 'American Bobtail', 
 'American Curl',
 'American Shorthair',
 'Bengal',
 'Birman',
 'Bombay',
 'British Shorthair',
 'Egyptian Mau',
 'Exotic Shorthair',
 'Maine Coon',
 'Manx',
 'Norwegian Forest',
 'Persian',
 'Ragdoll',
 'Russian Blue',
 'Scottish Fold',
 'Siamese',
 'Sphynx',
 'Turkish Angora']

In [ ]:
dict(zip(classes, np.round(pred[0], 4)))

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('catbreeds-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [ ]:
# !pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [11]:
pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 3.6 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install absl-py==0.10 
# !pip install flatbuffers==1.12.0 
# !pip install gast==0.3.3 
# !pip install grpcio==1.32.0 
!pip uninstall numpy==1.19.2 
##  !pip install numpy==1.19.2 
# !pip install six==1.15.0 
# !pip install tensorflow-estimator==2.4.0 typing-extensions==3.7.4 wrapt==1.12.1

     |████████████████████████████████| 127 kB 3.9 MB/s eta 0:00:01
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.0.0
    Uninstalling absl-py-1.0.0:
      Successfully uninstalled absl-py-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0 which is incompatible.
tensorflow 2.4.1 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.4.1 requires grpcio~=1.32.0, but you have grpcio 1.46.1 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.26.3 which is incompatible.
tensorflow 2.4.1 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.4.1 requires tensorflow-estimator<2.5.0,>=2.4.0, but you have tensorflow-estimator 2.6.0 which is incompatible.
tensorflow 2.4.1 requir

In [ ]:
!pip install tflite-runtime

In [1]:
# import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite

ImportError: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /srv/conda/envs/saturn/lib/python3.9/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so)

In [ ]:
interpreter = tflite.Interpreter(model_path='catbreeds-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
classes = ['Abyssinian', 
 'American Bobtail', 
 'American Curl',
 'American Shorthair',
 'Bengal',
 'Birman',
 'Bombay',
 'British Shorthair',
 'Egyptian Mau',
 'Exotic Shorthair',
 'Maine Coon',
 'Manx',
 'Norwegian Forest',
 'Persian',
 'Ragdoll',
 'Russian Blue',
 'Scottish Fold',
 'Siamese',
 'Sphynx',
 'Turkish Angora']

dict(zip(classes, np.round(pred[0], 4)))

***Remove TF dependency***

In [ ]:
from PIL import Image

In [ ]:
with Image.open(fullname) as img:
    img = img.resize((150, 150), Image.NEAREST)

In [ ]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [ ]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
classes = ['Abyssinian', 
 'American Bobtail', 
 'American Curl',
 'American Shorthair',
 'Bengal',
 'Birman',
 'Bombay',
 'British Shorthair',
 'Egyptian Mau',
 'Exotic Shorthair',
 'Maine Coon',
 'Manx',
 'Norwegian Forest',
 'Persian',
 'Ragdoll',
 'Russian Blue',
 'Scottish Fold',
 'Siamese',
 'Sphynx',
 'Turkish Angora']

dict(zip(classes, np.round(pred[0], 4)))

## Evaluate

# Communicate the Results